## 1. Lý thuyết về High-Utility Itemset Mining

### 1.1 Khái niệm cơ bản

**Utility** (giá trị/lợi ích) của một itemset trong một giao dịch được tính bằng:

$$U(X, T_k) = \sum_{x \in X} q(x, T_k) \times p(x)$$

Trong đó:
- $X$: Itemset (tập các sản phẩm)
- $T_k$: Giao dịch thứ k
- $q(x, T_k)$: Số lượng của item x trong giao dịch $T_k$ (internal utility)
- $p(x)$: Lợi nhuận/giá của item x (external utility)

**Tổng Utility** của itemset X trong toàn bộ database:

$$U(X) = \sum_{T_k \supseteq X} U(X, T_k)$$

### 1.2 Vấn đề với Downward Closure Property

Trong Frequent Pattern Mining, tính chất **Apriori** (downward closure) cho phép cắt tỉa hiệu quả:
- Nếu itemset X không phổ biến → tất cả superset của X cũng không phổ biến

**Nhưng với Utility**, tính chất này **KHÔNG** còn đúng:
- {A} có utility = 10
- {A, B} có thể có utility = 100 (do B có giá trị cao)

→ Cần sử dụng **Transaction-Weighted Utilization (TWU)** làm upper bound để cắt tỉa.

### 1.3 Transaction-Weighted Utilization (TWU)

$$TWU(X) = \sum_{T_k \supseteq X} TU(T_k)$$

Trong đó $TU(T_k)$ là tổng utility của giao dịch $T_k$.

**Tính chất quan trọng**: TWU có tính chất downward closure!
- Nếu $TWU(X) < minUtil$ → X và tất cả superset của X không phải High-Utility Itemset

## Parameters (for papermill)

In [ ]:
# PARAMETERS (for papermill)

# Đường dẫn dữ liệu
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"

# Đường dẫn lưu kết quả
OUTPUT_DIR = "data/processed"
HUI_OUTPUT_PATH = "data/processed/high_utility_itemsets.csv"

# ⚡ THAM SỐ TỐI ƯU HÓA TỐC ĐỘ ⚡
# Tăng ngưỡng utility để giảm số itemsets cần xử lý
MIN_UTILITY_PERCENT = 0.1 

# Hoặc dùng giá trị tuyệt đối (nếu > 0 sẽ ưu tiên dùng cái này)
MIN_UTILITY_ABSOLUTE = 0  # Đặt > 0 nếu muốn dùng ngưỡng tuyệt đối (ví dụ: 10000)

# Giảm độ dài tối đa để giảm số combinations
MAX_ITEMSET_LENGTH = 2

# Cột dữ liệu
INVOICE_COL = "InvoiceNo"
ITEM_COL = "Description"
QUANTITY_COL = "Quantity"
PRICE_COL = "UnitPrice"
TOTAL_COL = "TotalPrice"

# Bật/tắt các biểu đồ
PLOT_TOP_HUI = True
PLOT_COMPARISON = True
PLOT_UTILITY_DISTRIBUTION = True

# Số lượng top itemsets để hiển thị
TOP_N = 5

# 3 MÀU CHỦ ĐẠO (định nghĩa ở đây để Papermill không ghi đè)
COLOR_BLUE = '#3498db'    # Xanh dương - cho Frequency/Volume
COLOR_GREEN = '#2ecc71'   # Xanh lá cây - cho Utility/Profit  
COLOR_ORANGE = '#e67e22'  # Cam - cho Highlights/Important
COLOR_GRAY = '#95a5a6'    # Xám - cho các mục khác
COLOR_RED = '#e74c3c'     # Đỏ - cho cảnh báo

# 💡 GỢI Ý TỐI ƯU:
# - MIN_UTILITY_PERCENT: 0.001 (chậm) -> 0.005 (vừa) -> 0.01 (nhanh)
# - MAX_ITEMSET_LENGTH: 3 (chậm) -> 2 (nhanh) -> 1 (rất nhanh, chỉ single items)
# - Để chạy cực nhanh thử nghiệm: MIN_UTILITY_PERCENT=0.01, MAX_ITEMSET_LENGTH=2

## 2. Set up

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import warnings
from collections import defaultdict
from itertools import combinations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Xác định project_root linh hoạt
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("../..")
elif "Lab_PhatTrien_5315" in cwd:
    project_root = os.path.abspath(os.path.join(cwd, "..", ".."))
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

# Style cho biểu đồ
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

# 3 MÀU CHỦ ĐẠO
COLOR_BLUE = '#3498db'    # Xanh dương - cho Frequency/Volume
COLOR_GREEN = '#2ecc71'   # Xanh lá cây - cho Utility/Profit  
COLOR_ORANGE = '#e67e22'  # Cam - cho Highlights/Important

# Màu phụ
COLOR_GRAY = '#95a5a6'
COLOR_RED = '#e74c3c'

print(f"Project root: {project_root}")
print(f"\n🎨 Màu chủ đạo: Xanh dương ({COLOR_BLUE}), Xanh lá ({COLOR_GREEN}), Cam ({COLOR_ORANGE})")

## 3. Tải và chuẩn bị dữ liệu

In [ ]:
# Đọc dữ liệu đã làm sạch
data_path = os.path.join(project_root, CLEANED_DATA_PATH)
df = pd.read_csv(data_path, parse_dates=["InvoiceDate"])

print("=== Thông tin dữ liệu ===")
print(f"- Số giao dịch (dòng): {df.shape[0]:,}")
print(f"- Số cột: {df.shape[1]}")
print(f"- Số hoá đơn duy nhất: {df[INVOICE_COL].nunique():,}")
print(f"- Số sản phẩm duy nhất: {df[ITEM_COL].nunique():,}")

df.head()

In [ ]:
# Kiểm tra thông tin utility
print("=== Thống kê Utility (TotalPrice) ===")
print(df[TOTAL_COL].describe())

total_utility = df[TOTAL_COL].sum()
print(f"\nTổng Utility (Doanh thu) của toàn bộ database: £{total_utility:,.2f}")

## 📊 Biểu đồ 1: Tổng quan dữ liệu

In [ ]:
# BIỂU ĐỒ 1: Tổng quan dữ liệu với 3 màu chủ đạo
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1.1 Phân phối TotalPrice (Utility)
ax1 = axes[0, 0]
df[TOTAL_COL].clip(upper=df[TOTAL_COL].quantile(0.95)).hist(
    bins=50, ax=ax1, color=COLOR_GREEN, edgecolor='white', alpha=0.8
)
ax1.axvline(df[TOTAL_COL].mean(), color=COLOR_ORANGE, linestyle='--', linewidth=2, label=f'Mean: £{df[TOTAL_COL].mean():.2f}')
ax1.axvline(df[TOTAL_COL].median(), color=COLOR_BLUE, linestyle='--', linewidth=2, label=f'Median: £{df[TOTAL_COL].median():.2f}')
ax1.set_title('📈 Phân phối Utility (TotalPrice)', fontsize=12, fontweight='bold')
ax1.set_xlabel('TotalPrice (£)')
ax1.set_ylabel('Số giao dịch')
ax1.legend()

# 1.2 Top 10 sản phẩm theo Utility
ax2 = axes[0, 1]
top_utility = df.groupby(ITEM_COL)[TOTAL_COL].sum().nlargest(10)
colors_top = [COLOR_GREEN if i < 3 else COLOR_BLUE for i in range(len(top_utility))]
ax2.barh(range(len(top_utility)), top_utility.values, color=colors_top)
ax2.set_yticks(range(len(top_utility)))
ax2.set_yticklabels([name[:30] + '...' if len(name) > 30 else name for name in top_utility.index], fontsize=9)
ax2.invert_yaxis()
ax2.set_title('💰 Top 10 Sản phẩm theo Utility', fontsize=12, fontweight='bold')
ax2.set_xlabel('Tổng Utility (£)')

# 1.3 Top 10 sản phẩm theo Frequency
ax3 = axes[1, 0]
top_freq = df.groupby(ITEM_COL)[INVOICE_COL].nunique().nlargest(10)
colors_freq = [COLOR_BLUE if i < 3 else COLOR_GREEN for i in range(len(top_freq))]
ax3.barh(range(len(top_freq)), top_freq.values, color=colors_freq)
ax3.set_yticks(range(len(top_freq)))
ax3.set_yticklabels([name[:30] + '...' if len(name) > 30 else name for name in top_freq.index], fontsize=9)
ax3.invert_yaxis()
ax3.set_title('🔢 Top 10 Sản phẩm theo Frequency', fontsize=12, fontweight='bold')
ax3.set_xlabel('Số lần xuất hiện')

# 1.4 So sánh tổng quan
ax4 = axes[1, 1]
metrics = ['Tổng Giao dịch', 'Số Hoá đơn', 'Số Sản phẩm', 'Tổng Utility (x1000£)']
values = [
    df.shape[0],
    df[INVOICE_COL].nunique(),
    df[ITEM_COL].nunique(),
    total_utility / 1000
]
colors_bar = [COLOR_BLUE, COLOR_GREEN, COLOR_ORANGE, COLOR_GREEN]
bars = ax4.bar(metrics, values, color=colors_bar, edgecolor='white', linewidth=2)
ax4.set_title('📋 Tổng quan Database', fontsize=12, fontweight='bold')
ax4.set_ylabel('Số lượng')

# Thêm giá trị trên thanh
for bar, val in zip(bars, values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.02,
             f'{val:,.0f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.suptitle('🔍 TỔNG QUAN DỮ LIỆU BÁN HÀNG', fontsize=14, fontweight='bold', y=1.02)
plt.show()

## 4. Triển khai High-Utility Itemset Mining

### 4.1 Class HighUtilityMiner

In [ ]:
class HighUtilityMiner:
    """
    High-Utility Itemset Mining using TWU-based approach.
    
    Thuật toán này sử dụng Transaction-Weighted Utilization (TWU) 
    làm upper bound để cắt tỉa không gian tìm kiếm hiệu quả.
    """
    
    def __init__(self, df, invoice_col, item_col, quantity_col, utility_col):
        """
        Khởi tạo HighUtilityMiner.
        
        Args:
            df: DataFrame chứa dữ liệu giao dịch
            invoice_col: Tên cột mã hoá đơn
            item_col: Tên cột sản phẩm
            quantity_col: Tên cột số lượng
            utility_col: Tên cột utility (doanh thu/lợi nhuận)
        """
        self.df = df.copy()
        self.invoice_col = invoice_col
        self.item_col = item_col
        self.quantity_col = quantity_col
        self.utility_col = utility_col
        
        # Chuẩn bị dữ liệu
        self._prepare_data()
        
    def _prepare_data(self):
        """Chuẩn bị dữ liệu cho HUIM."""
        # Tính utility cho mỗi item trong mỗi giao dịch
        self.transactions = {}
        self.transaction_utilities = {}  # TU(T_k)
        self.item_utilities = defaultdict(float)  # Tổng utility của mỗi item
        self.item_twu = defaultdict(float)  # TWU của mỗi item
        
        for invoice, group in self.df.groupby(self.invoice_col):
            # Lưu các item và utility trong giao dịch
            items_utilities = {}
            for _, row in group.iterrows():
                item = row[self.item_col]
                utility = row[self.utility_col]
                items_utilities[item] = items_utilities.get(item, 0) + utility
            
            self.transactions[invoice] = items_utilities
            tu = sum(items_utilities.values())  # Transaction Utility
            self.transaction_utilities[invoice] = tu
            
            # Cập nhật TWU cho mỗi item
            for item in items_utilities:
                self.item_twu[item] += tu
                self.item_utilities[item] += items_utilities[item]
        
        self.total_utility = sum(self.transaction_utilities.values())
        print(f"Đã chuẩn bị {len(self.transactions):,} giao dịch")
        print(f"Tổng utility của database: £{self.total_utility:,.2f}")
    
    def mine_high_utility_itemsets(self, min_utility, max_length=3):
        """
        Khai thác High-Utility Itemsets sử dụng TWU pruning.
        
        Args:
            min_utility: Ngưỡng utility tối thiểu
            max_length: Độ dài tối đa của itemset
            
        Returns:
            DataFrame chứa các High-Utility Itemsets
        """
        print(f"\n=== Mining High-Utility Itemsets ===")
        print(f"Min Utility: £{min_utility:,.2f}")
        print(f"Max Length: {max_length}")
        
        start_time = time.time()
        
        # Bước 1: Lọc các item có TWU >= min_utility
        promising_items = [
            item for item, twu in self.item_twu.items() 
            if twu >= min_utility
        ]
        print(f"Số item có TWU >= min_utility: {len(promising_items):,}")
        
        # Bước 2: Tính utility thực tế cho 1-itemsets
        hui_results = []
        
        for item in promising_items:
            utility = self.item_utilities[item]
            if utility >= min_utility:
                hui_results.append({
                    'itemset': frozenset([item]),
                    'utility': utility,
                    'length': 1,
                    'twu': self.item_twu[item]
                })
        
        print(f"1-itemsets HUI: {len(hui_results):,}")
        
        # Bước 3: Sinh k-itemsets (k >= 2)
        if max_length >= 2:
            # Tạo itemset TWU map cho việc generate candidates
            for k in range(2, max_length + 1):
                k_itemsets = self._generate_k_itemsets(
                    promising_items, k, min_utility
                )
                hui_results.extend(k_itemsets)
                print(f"{k}-itemsets HUI: {len(k_itemsets):,}")
                
                if len(k_itemsets) == 0:
                    break
        
        elapsed = time.time() - start_time
        print(f"\nThời gian chạy: {elapsed:.2f} giây")
        print(f"Tổng số High-Utility Itemsets: {len(hui_results):,}")
        
        # Tạo DataFrame kết quả
        if hui_results:
            df_hui = pd.DataFrame(hui_results)
            df_hui['itemset_str'] = df_hui['itemset'].apply(
                lambda x: ', '.join(sorted(list(x)))
            )
            df_hui['utility_percent'] = (df_hui['utility'] / self.total_utility * 100)
            df_hui = df_hui.sort_values('utility', ascending=False).reset_index(drop=True)
            return df_hui
        else:
            return pd.DataFrame()
    
    def _generate_k_itemsets(self, items, k, min_utility):
        """
        Sinh và đánh giá k-itemsets.
        
        Args:
            items: Danh sách các item tiềm năng
            k: Độ dài itemset
            min_utility: Ngưỡng utility tối thiểu
            
        Returns:
            List các k-itemsets có utility >= min_utility
        """
        results = []
        
        # Sinh tất cả k-itemsets từ promising items
        for combo in combinations(sorted(items), k):
            itemset = frozenset(combo)
            
            # Tính TWU của itemset
            twu = self._calculate_twu(itemset)
            
            # Pruning: nếu TWU < min_utility, bỏ qua
            if twu < min_utility:
                continue
            
            # Tính utility thực tế
            utility = self._calculate_utility(itemset)
            
            if utility >= min_utility:
                results.append({
                    'itemset': itemset,
                    'utility': utility,
                    'length': k,
                    'twu': twu
                })
        
        return results
    
    def _calculate_twu(self, itemset):
        """Tính Transaction-Weighted Utilization của itemset."""
        twu = 0
        for invoice, items_utils in self.transactions.items():
            if itemset.issubset(set(items_utils.keys())):
                twu += self.transaction_utilities[invoice]
        return twu
    
    def _calculate_utility(self, itemset):
        """Tính utility thực tế của itemset."""
        total_utility = 0
        for invoice, items_utils in self.transactions.items():
            if itemset.issubset(set(items_utils.keys())):
                # Cộng utility của các item trong itemset
                for item in itemset:
                    total_utility += items_utils.get(item, 0)
        return total_utility
    
    def get_item_statistics(self):
        """Lấy thống kê về utility của từng item."""
        stats = []
        for item in self.item_utilities:
            stats.append({
                'item': item,
                'utility': self.item_utilities[item],
                'twu': self.item_twu[item],
                'utility_percent': self.item_utilities[item] / self.total_utility * 100
            })
        
        df_stats = pd.DataFrame(stats)
        df_stats = df_stats.sort_values('utility', ascending=False).reset_index(drop=True)
        return df_stats

### 4.2 Khai thác High-Utility Itemsets

In [ ]:
# Khởi tạo miner
hui_miner = HighUtilityMiner(
    df=df,
    invoice_col=INVOICE_COL,
    item_col=ITEM_COL,
    quantity_col=QUANTITY_COL,
    utility_col=TOTAL_COL
)

In [ ]:
# Tính ngưỡng utility
if MIN_UTILITY_ABSOLUTE > 0:
    min_utility = MIN_UTILITY_ABSOLUTE
else:
    min_utility = hui_miner.total_utility * MIN_UTILITY_PERCENT

print(f"Ngưỡng Min Utility: £{min_utility:,.2f}")
print(f"(Tương đương {min_utility/hui_miner.total_utility*100:.4f}% tổng doanh thu)")

In [ ]:
# Khai thác High-Utility Itemsets
df_hui = hui_miner.mine_high_utility_itemsets(
    min_utility=min_utility,
    max_length=MAX_ITEMSET_LENGTH
)

print("\n=== Top High-Utility Itemsets ===")
if not df_hui.empty:
    display_cols = ['itemset_str', 'utility', 'utility_percent', 'length']
    print(df_hui[display_cols].head(TOP_N).to_string())

## 5. Phân tích và So sánh

### 5.1 Thống kê utility theo item

In [ ]:
# Lấy thống kê item
df_item_stats = hui_miner.get_item_statistics()

print("=== Top 20 Items theo Utility ===")
print(df_item_stats.head(20).to_string())

In [ ]:
# So sánh với Frequency
# Tính frequency của mỗi item
item_freq = df.groupby(ITEM_COL)[INVOICE_COL].nunique().reset_index()
item_freq.columns = ['item', 'frequency']
item_freq['frequency_percent'] = item_freq['frequency'] / df[INVOICE_COL].nunique() * 100

# Merge với utility stats
df_comparison = df_item_stats.merge(item_freq, on='item', how='left')
df_comparison = df_comparison.sort_values('utility', ascending=False)

print("\n=== So sánh Utility vs Frequency (Top 20 theo Utility) ===")
print(df_comparison[['item', 'utility', 'utility_percent', 'frequency', 'frequency_percent']].head(20).to_string())

### 5.2 Trực quan hóa

In [ ]:
if PLOT_TOP_HUI and not df_hui.empty:
    # Plot Top High-Utility Itemsets
    fig, ax = plt.subplots(figsize=(14, 8))
    
    top_hui = df_hui.head(TOP_N).copy()
    colors = sns.color_palette('viridis', len(top_hui))
    
    bars = ax.barh(
        range(len(top_hui)), 
        top_hui['utility'],
        color=colors
    )
    
    ax.set_yticks(range(len(top_hui)))
    ax.set_yticklabels(top_hui['itemset_str'], fontsize=9)
    ax.invert_yaxis()
    
    ax.set_xlabel('Utility (£)', fontsize=12)
    ax.set_title(f'Top {TOP_N} High-Utility Itemsets', fontsize=14, fontweight='bold')
    
    # Thêm giá trị trên thanh
    for i, (bar, utility) in enumerate(zip(bars, top_hui['utility'])):
        ax.text(
            bar.get_width() + 100, bar.get_y() + bar.get_height()/2,
            f'£{utility:,.0f}',
            va='center', fontsize=9
        )
    
    plt.tight_layout()
    plt.show()

In [ ]:
if PLOT_COMPARISON:
    # Scatter plot: Utility vs Frequency
    fig = px.scatter(
        df_comparison.head(100),
        x='frequency_percent',
        y='utility_percent',
        hover_name='item',
        size='utility',
        color='utility',
        color_continuous_scale='Viridis',
        title='So sánh Utility vs Frequency của các sản phẩm',
        labels={
            'frequency_percent': 'Frequency (%)',
            'utility_percent': 'Utility (%)',
            'utility': 'Utility (£)'
        }
    )
    
    fig.update_layout(
        width=900,
        height=600,
        font=dict(size=12)
    )
    
    fig.show()

In [ ]:
if PLOT_UTILITY_DISTRIBUTION:
    # Phân phối utility theo độ dài itemset
    if not df_hui.empty and 'length' in df_hui.columns:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Số lượng HUI theo độ dài
        length_counts = df_hui['length'].value_counts().sort_index()
        axes[0].bar(length_counts.index, length_counts.values, color='steelblue')
        axes[0].set_xlabel('Độ dài Itemset')
        axes[0].set_ylabel('Số lượng HUI')
        axes[0].set_title('Phân phối High-Utility Itemsets theo độ dài')
        
        # Tổng utility theo độ dài
        utility_by_length = df_hui.groupby('length')['utility'].sum()
        axes[1].bar(utility_by_length.index, utility_by_length.values, color='coral')
        axes[1].set_xlabel('Độ dài Itemset')
        axes[1].set_ylabel('Tổng Utility (£)')
        axes[1].set_title('Tổng Utility theo độ dài Itemset')
        
        plt.tight_layout()
        plt.show()

### 5.3 Phát hiện "Hidden Gems" - Sản phẩm giá trị cao nhưng ít phổ biến

In [ ]:
# Tìm các sản phẩm có utility cao nhưng frequency thấp
# Đây là những "hidden gems" mà Frequent Pattern Mining có thể bỏ lỡ

median_freq = df_comparison['frequency_percent'].median()
median_util = df_comparison['utility_percent'].median()

hidden_gems = df_comparison[
    (df_comparison['frequency_percent'] < median_freq) &
    (df_comparison['utility_percent'] > median_util * 2)
].head(20)

print("=== HIDDEN GEMS: Sản phẩm có Utility cao nhưng Frequency thấp ===")
print("(Những sản phẩm này có thể bị bỏ lỡ nếu chỉ dùng Frequent Pattern Mining)")
print()
if not hidden_gems.empty:
    print(hidden_gems[['item', 'utility', 'utility_percent', 'frequency', 'frequency_percent']].to_string())
else:
    print("Không tìm thấy hidden gems với ngưỡng hiện tại.")

## 6. So sánh với Frequent Itemsets (từ Apriori)

### Sự khác biệt tư duy giữa "Frequent" và "High-Utility"

In [ ]:
# Tính support cho các item
n_transactions = df[INVOICE_COL].nunique()

df_comparison['support'] = df_comparison['frequency'] / n_transactions

# Phân loại items
df_comparison['category'] = 'Khác'

# High Utility, High Frequency
df_comparison.loc[
    (df_comparison['utility_percent'] > df_comparison['utility_percent'].quantile(0.75)) &
    (df_comparison['frequency_percent'] > df_comparison['frequency_percent'].quantile(0.75)),
    'category'
] = 'Stars (High Utility + High Freq)'

# High Utility, Low Frequency
df_comparison.loc[
    (df_comparison['utility_percent'] > df_comparison['utility_percent'].quantile(0.75)) &
    (df_comparison['frequency_percent'] <= df_comparison['frequency_percent'].quantile(0.25)),
    'category'
] = 'Hidden Gems (High Utility, Low Freq)'

# Low Utility, High Frequency
df_comparison.loc[
    (df_comparison['utility_percent'] <= df_comparison['utility_percent'].quantile(0.25)) &
    (df_comparison['frequency_percent'] > df_comparison['frequency_percent'].quantile(0.75)),
    'category'
] = 'Volume Drivers (Low Utility, High Freq)'

print("=== Phân loại sản phẩm ===")
print(df_comparison['category'].value_counts())

In [ ]:
# Visualize phân loại
fig = px.scatter(
    df_comparison,
    x='frequency_percent',
    y='utility_percent',
    color='category',
    hover_name='item',
    title='Ma trận Utility-Frequency: Phân loại sản phẩm',
    labels={
        'frequency_percent': 'Frequency (%)',
        'utility_percent': 'Utility (%)',
        'category': 'Phân loại'
    },
    color_discrete_map={
        'Stars (High Utility + High Freq)': '#2ecc71',
        'Hidden Gems (High Utility, Low Freq)': '#e74c3c',
        'Volume Drivers (Low Utility, High Freq)': '#3498db',
        'Khác': '#95a5a6'
    }
)

fig.update_layout(
    width=1000,
    height=700,
    font=dict(size=12)
)

# Thêm đường tham chiếu
fig.add_hline(
    y=df_comparison['utility_percent'].quantile(0.75),
    line_dash="dash", line_color="gray",
    annotation_text="75th percentile Utility"
)
fig.add_vline(
    x=df_comparison['frequency_percent'].quantile(0.75),
    line_dash="dash", line_color="gray",
    annotation_text="75th percentile Frequency"
)

fig.show()

## 7. Lưu kết quả

In [ ]:
# Lưu High-Utility Itemsets
output_path = os.path.join(project_root, HUI_OUTPUT_PATH)
os.makedirs(os.path.dirname(output_path), exist_ok=True)

if not df_hui.empty:
    # Convert frozenset to string for saving
    df_hui_save = df_hui.copy()
    df_hui_save['itemset'] = df_hui_save['itemset'].apply(lambda x: str(set(x)))
    df_hui_save.to_csv(output_path, index=False)
    print(f"Đã lưu High-Utility Itemsets: {output_path}")
else:
    print("Không có High-Utility Itemsets để lưu.")

## 7. Lưu kết quả

In [ ]:
# Lưu High-Utility Itemsets
output_path = os.path.join(project_root, HUI_OUTPUT_PATH)
os.makedirs(os.path.dirname(output_path), exist_ok=True)

if not df_hui.empty:
    # Convert frozenset to string for saving
    df_hui_save = df_hui.copy()
    df_hui_save['itemset'] = df_hui_save['itemset'].apply(lambda x: str(set(x)))
    df_hui_save.to_csv(output_path, index=False)
    print(f"Đã lưu High-Utility Itemsets: {output_path}")
else:
    print("Không có High-Utility Itemsets để lưu.")

In [ ]:
# Lưu thống kê item comparison
comparison_path = os.path.join(project_root, OUTPUT_DIR, 'item_utility_frequency_comparison.csv')
df_comparison.to_csv(comparison_path, index=False)
print(f"Đã lưu so sánh Utility vs Frequency: {comparison_path}")

## 8. Kết luận

### 8.1 Sự khác biệt tư duy giữa Frequent và High-Utility Mining

| Frequent Pattern Mining | High-Utility Itemset Mining |
|------------------------|-----------------------------|
| Tìm sản phẩm **bán chạy** | Tìm sản phẩm **sinh lời** |
| Tối ưu **số lượng giao dịch** | Tối ưu **giá trị doanh thu** |
| Dễ implement (Apriori property) | Phức tạp hơn (cần TWU bound) |
| Phù hợp cho cross-selling | Phù hợp cho **tối ưu lợi nhuận** |

### 8.2 Insights từ phân tích

1. **Hidden Gems**: Các sản phẩm có utility cao nhưng frequency thấp - đây là những sản phẩm có thể bị bỏ lỡ nếu chỉ dùng Frequent Pattern Mining.

2. **Volume Drivers**: Các sản phẩm bán chạy nhưng margin thấp - quan trọng để duy trì traffic nhưng không phải nguồn lợi nhuận chính.

3. **Stars**: Sản phẩm vừa bán chạy vừa sinh lời cao - cần ưu tiên quảng bá và đảm bảo inventory.

### 8.3 Đề xuất kinh doanh

- **Với Hidden Gems**: Tăng visibility, marketing để chuyển thành Stars
- **Với Volume Drivers**: Sử dụng làm sản phẩm "loss leader" để thu hút khách
- **Với Stars**: Đảm bảo inventory, tạo bundle deals
- **Cross-selling**: Kết hợp High-Utility Itemsets để gợi ý sản phẩm tối ưu doanh thu

---

## Tài liệu tham khảo

1. Liu, Y., Liao, W., & Choudhary, A. (2005). A two-phase algorithm for fast discovery of high utility itemsets. *PAKDD 2005*.

2. Fournier-Viger, P., et al. (2014). FHM: Faster high-utility itemset mining using estimated utility co-occurrence pruning. *ISMIS 2014*.

3. Gan, W., et al. (2021). A survey of utility-oriented pattern mining. *IEEE Transactions on Knowledge and Data Engineering*.

4. Thư viện SPMF: http://www.philippe-fournier-viger.com/spmf/ (Java library for pattern mining)